# analysis

## ToDo

- __Importing more than one data json__
- How to group by subjects models and movements
- Model specific confusion rate
- How to manage or represent all this data
- ...

## data structure

...

In [15]:
import numpy as np
import pandas as pd
import json, os

In [16]:
path_to_json = '/users/hieu/git/turing_test_walking/data'

json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
first_file = json_files.pop()
with open(first_file,'r') as f:
    one = json.loads(f.read())
data = pd.json_normalize(one)

In [17]:
json_files

['experiment_data_undefined_v9frg7u7uvw37fg.json',
 'experiment_data_3973_w75wd3tdpjxptzo.json',
 'experiment_data_3973_bz18s5macfxk74y.json',
 'experiment_data_undefined_zkbuomnbzej51l7.json',
 'experiment_data_3973_arpj52d3xdw15s5.json',
 'experiment_data_undefined_r6vg0soyf6xlcpt.json',
 'experiment_data_undefined_b2pjb3quqspdbq8.json',
 'experiment_data_undefined_x2kwukss3paod9f.json',
 'experiment_data_3973_qcsx88zcoj4qgcu.json',
 'experiment_data_THNS_pvmkyoht9kcvsng.json',
 'experiment_data_3973_dfhhna3z9j2v2s8.json',
 'data.json',
 'experiment_data_undefined_72z69lw70zafxgx.json',
 'experiment_data_undefined_f1gg8lj4h87tu1r.json',
 'experiment_data_undefined_dzldq3v3no0348r.json',
 'experiment_data_THNS_lynundkl3z6g33s.json',
 'experiment_data_undefined_jt1xjnxf1o0mhd5.json',
 'experiment_data_THNS_kknnbzndv252gun.json',
 'experiment_data_undefined_gl9u2qvu2cmvtjq.json',
 'experiment_data_undefined_onzokuqujwnax3t.json']

In [18]:
for file in json_files:
    with open(file,'r') as f:
        one = json.loads(f.read())
    load = pd.json_normalize(one)
    data = pd.concat([data,load])

In [19]:
data

,rt,stimulus,button_pressed,trial_type,trial_index,time_elapsed,internal_node_id,subject,view_history,key_press,test_part,test_stimulus.position,test_stimulus.left,test_stimulus.right,test_stimulus.att_check,sona
0,973.000,<p>Welcome!<br> Please choose your language to...,1,html-button-response,0,980,0.0-0.0,bt4quvsv7wdsku4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2206.000,NaN,NaN,instructions,1,3188,0.0-1.0-0.0,bt4quvsv7wdsku4,"[{""page_index"":0,""viewing_time"":515},{""page_in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,785.000,<p>Bereit?</p>,0,html-button-response,2,3976,0.0-3.0,bt4quvsv7wdsku4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"<div style=""font-size:60px;"">+</div>",NaN,html-keyboard-response,3,5981,0.0-4.0,bt4quvsv7wdsku4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,training,4,9769,0.0-5.0,bt4quvsv7wdsku4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,1274.935,<p>Welche Sequenz haben Sie als natürlicher em...,1,html-button-response,42,245523,0.0-7.0-3.8,onzokuqujwnax3t,NaN,NaN,response,0.0,bvh/return-bottle-training2-lines.txt,bvh/vr_prediction_models/model(tmp)-dataset(re...,False,NaN
43,981.395,<p>Bereit?</p>,0,html-button-response,43,246508,0.0-7.0-0.9,onzokuqujwnax3t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,NaN,"<div style=""font-size:60px;"">+</div>",NaN,html-keyboard-response,44,248512,0.0-7.0-1.9,onzokuqujwnax3t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,NaN,NaN,turing-test,45,254185,0.0-7.0-2.9,onzokuqujwnax3t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


  ## parsing

In [20]:
df = data[data['test_part'] == 'response'].reset_index()

df['button_pressed'] = df['button_pressed'].astype(int)
df['test_stimulus.position'] = df['test_stimulus.position'].astype(int)

df = df.rename(columns={"test_stimulus.left": "left",
                        "test_stimulus.right": "right",
                        "test_stimulus.att_check": "att_check",
                        "test_stimulus.position": "position",
                        "button_pressed":"pressed"})

In [21]:
df = df[['subject','rt','pressed','position',
    'left','right','att_check']]

In [22]:
df

,subject,rt,pressed,position,left,right,att_check
0,bt4quvsv7wdsku4,5540.000,0,0,bvh/pass-bottle-hold-training0-lines.txt,bvh/vr_prediction_models/model(dmp)-dataset(pa...,False
1,bt4quvsv7wdsku4,792.000,0,0,bvh/pass-bottle-hold-training1-lines.txt,bvh/vr_prediction_models/model(tmp)-dataset(pa...,False
2,bt4quvsv7wdsku4,2076.000,1,1,bvh/vr_prediction_models/model(vcgpdm)-dataset...,bvh/return-bottle-hold-training3-lines.txt,False
3,bt4quvsv7wdsku4,1266.000,1,1,bvh/vr_prediction_models/model(tmp)-dataset(pa...,bvh/pass-bottle-training0-lines.txt,False
4,bt4quvsv7wdsku4,2022.000,1,0,bvh/return-bottle-hold-training4-lines.txt,bvh/vr_prediction_models/model(vcgpdm)-dataset...,False
...,...,...,...,...,...,...,...
705,onzokuqujwnax3t,1723.110,1,1,bvh/vr_prediction_models/model(tmp)-dataset(re...,bvh/return-bottle-training4-lines.txt,False
706,onzokuqujwnax3t,1248.325,0,0,bvh/pass-bottle-training4-lines.txt,bvh/vr_prediction_models/model(dmp)-dataset(pa...,True
707,onzokuqujwnax3t,1372.765,0,1,bvh/vr_prediction_models/model(vcgpdm)-dataset...,bvh/pass-bottle-training2-lines.txt,False
708,onzokuqujwnax3t,1274.935,1,0,bvh/return-bottle-training2-lines.txt,bvh/vr_prediction_models/model(tmp)-dataset(re...,False


In [23]:
# without copy() there is an error when replacing strings
d = df.copy()

In [24]:
# removing unnecessary strings
d['left'] = d['left'].str.replace('bvh/vr_prediction_models/','')
d['left'] = d['left'].str.replace('bvh/','')
d['left'] = d['left'].str.replace('/final-lines.txt','')
d['left'] = d['left'].str.replace('-lines.txt','')
d['right'] = d['right'].str.replace('bvh/vr_prediction_models/','')
d['right'] = d['right'].str.replace('bvh/','')
d['right'] = d['right'].str.replace('/final-lines.txt','')
d['right'] = d['right'].str.replace('-lines.txt','')

In [25]:
num = 0
model = []
for i in d['position']:
    if i == 1:
        model.append(d['left'][num])
    else:
        model.append(d['right'][num])
    num += 1
d['model'] = model

In [26]:
d

,subject,rt,pressed,position,left,right,att_check,model
0,bt4quvsv7wdsku4,5540.000,0,0,pass-bottle-hold-training0,model(dmp)-dataset(pass-bottle-hold)-npsi(100)...,False,model(dmp)-dataset(pass-bottle-hold)-npsi(100)...
1,bt4quvsv7wdsku4,792.000,0,0,pass-bottle-hold-training1,model(tmp)-dataset(pass-bottle-hold)-numprim(1...,False,model(tmp)-dataset(pass-bottle-hold)-numprim(1...
2,bt4quvsv7wdsku4,2076.000,1,1,model(vcgpdm)-dataset(return-bottle-hold)-mode...,return-bottle-hold-training3,False,model(vcgpdm)-dataset(return-bottle-hold)-mode...
3,bt4quvsv7wdsku4,1266.000,1,1,model(tmp)-dataset(pass-bottle)-numprim(4)-hol...,pass-bottle-training0,False,model(tmp)-dataset(pass-bottle)-numprim(4)-hol...
4,bt4quvsv7wdsku4,2022.000,1,0,return-bottle-hold-training4,model(vcgpdm)-dataset(return-bottle-hold)-mode...,False,model(vcgpdm)-dataset(return-bottle-hold)-mode...
...,...,...,...,...,...,...,...,...
705,onzokuqujwnax3t,1723.110,1,1,model(tmp)-dataset(return-bottle)-numprim(14)-...,return-bottle-training4,False,model(tmp)-dataset(return-bottle)-numprim(14)-...
706,onzokuqujwnax3t,1248.325,0,0,pass-bottle-training4,model(dmp)-dataset(pass-bottle)-npsi(70)-hold(4),True,model(dmp)-dataset(pass-bottle)-npsi(70)-hold(4)
707,onzokuqujwnax3t,1372.765,0,1,model(vcgpdm)-dataset(pass-bottle)-mode(ELBO)-...,pass-bottle-training2,False,model(vcgpdm)-dataset(pass-bottle)-mode(ELBO)-...
708,onzokuqujwnax3t,1274.935,1,0,return-bottle-training2,model(tmp)-dataset(return-bottle)-numprim(10)-...,False,model(tmp)-dataset(return-bottle)-numprim(10)-...


## Analysis
### Individual confusion rate over all models
#### with all attention checks included

In [27]:
arr = list(zip(d['pressed'],d['position']))
right = 0
for i in arr:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(arr) - right)/len(arr)
print(confusion_rate)

0.28450704225352114


#### without attention checks

In [28]:
no_check = d[d['att_check']==False]

In [29]:
arr_no_check = list(zip(no_check['pressed'],no_check['position']))
right = 0
for i in arr_no_check:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(arr_no_check) - right)/len(arr_no_check)
print(confusion_rate)

0.3042121684867395


### Checking attention
#### if confusion rate is zero -> good

In [30]:
check_att = d[d['att_check']==True]

In [31]:
arr_check = list(zip(check_att['pressed'],check_att['position']))
right = 0
for i in arr_check:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(arr_check) - right)/len(arr_check)
print(confusion_rate)

0.10144927536231885


In [32]:
for i in d:
        print(d[i].shape)

(710,)
(710,)
(710,)
(710,)
(710,)
(710,)
(710,)
(710,)


In [33]:
df = []
for item in json_files: 
    with open(item,'r') as f: 
        load = json.loads(f.read())
        df.append(pd.json_normalize(load))

In [34]:
df

[        rt                                           stimulus button_pressed  \
 0   1528.0  <p>Welcome!<br> Please choose your language to...              1   
 1   2062.0                                                NaN            NaN   
 2    578.0                                     <p>Bereit?</p>              0   
 3      NaN               <div style="font-size:60px;">+</div>            NaN   
 4      NaN                                                NaN            NaN   
 5   1002.0  <p>Welche Sequenz haben Sie als natürlicher em...              0   
 6    868.0  <p>Das Experiment startet nun.</p><p> Bitte dr...              0   
 7    769.0                                     <p>Bereit?</p>              0   
 8      NaN               <div style="font-size:60px;">+</div>            NaN   
 9      NaN                                                NaN            NaN   
 10  1277.0  <p>Welche Sequenz haben Sie als natürlicher em...              0   
 11   696.0                 